In [ ]:
#!pip install xgboost==1.6.1

In [ ]:
import hashlib
import os
import tarfile
import zipfile
import requests
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn import metrics
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import scipy.stats
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import scipy.stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import stats

from sklearn.pipeline import make_pipeline
import time
from itertools import cycle
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from scipy import stats
from sklearn.preprocessing import FunctionTransformer

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils import data
import torch.optim as optim
from tqdm.notebook import tqdm
from scipy.stats import pearsonr

import tensorflow as tf
from tensorflow import keras

DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

In [ ]:
def download(name, cache_dir=os.path.join('..', 'data')):
    """Download a file inserted into DATA_HUB, return the local filename."""
    assert name in DATA_HUB, f"{name} does not exist in {DATA_HUB}."
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                  break
            sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
          return fname  # Hit cache
    print(f'Downloading {fname} from {url}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
      f.write(r.content)
    return fname

def download_extract(name, folder=None):
    """Download and extract a zip/tar file."""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, 'Only zip/tar files can be extracted.'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir

def download_all():
    """Download all files in the DATA_HUB."""
    for name in DATA_HUB:
        download(name)

DATA_HUB['kaggle_house_train'] = (
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')
DATA_HUB['kaggle_house_test'] = (
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

In [ ]:
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [ ]:
class clean_data:

  def __init__(self, df, target):
        self.df = df
        self.target = target

  def remove_outliers_z_test(self, df, target):
    # Handle Outliers in the data
    #Define outliers as Z-score greater than 3
    #z-score = (x = mean)/sq
    z = np.abs(stats.zscore(df[target]))
    threshold = 3
    out = np.where(z > threshold)

    print(len(out[0]))
    print(df.shape)
    #train_data.iloc[out]

  def remove_outliers_iqr(self, df, target):
    Q1 = df[target].quantile(q = 0.25) 
    Q3 = df[target].quantile(q = 0.75)
    IQR = Q3 - Q1

    # Sort DF by the 'SalePrice' col
    sorted_values = df.sort_values(target) 

    # Take the median of our now sorted 'SalePrice'
    median = sorted_values[target].median() 

    # Apply IQR to the 'SalePrice' col
    q1 = sorted_values[target].quantile(q = 0.25) 
    q3 = sorted_values[target].quantile(q = 0.75) 
    iqr = q3 - q1

    Inner_bound1 = q1 - (iqr * 1.5) 
    print(f'Inner Boundary 1 = {Inner_bound1}')

    Inner_bound2 = q3 + (iqr * 1.5)  
    print(f'Inner Boundary 2 = {Inner_bound2}')

    Outer_bound1 = q1 - (iqr * 3)    
    print(f'Outer Boundary 1 = {Outer_bound1}')

    Outer_bound2 = q3 + (iqr * 3)   
    print(f'Outer Boundary 2 = {Outer_bound2}')

    # SEt this as the outer boundary to remove all outliers beyond it
    self.Df = df[df[target] <= Outer_bound2]

    # CHECKS
    print(f'Data before Removing Outliers = {df.shape}')
    print(f'Data after Removing Outliers = {self.Df.shape}')
    print(f'Number of Outliers removed = {df.shape[0] - self.Df.shape[0]}')

    return self.Df

  def feature_selection(self,df, target):

      #self.train_data = self.remove_outliers_iqr(df, target)

      corr = df.corr().reset_index()[['index', target]]
      #train_data.corr().reset_index()

      interested_cols=[]

      for item in corr.values:
          if item[1:2]>=0.15 or item[1:2]<=-0.15:
              interested_cols.append(item[0:1][0])

      #interested_cols.remove('SalePrice')

      return interested_cols

In [ ]:
class My_encoder:
    '''
    This class creates a pipline which is used for applying SimpleImputer, MinMaxScaler & OneHotEncoder.
    '''

    def __init__(self):
        pass
        
    def encoder_pipeline(self, numerical_features, categorical_features):
        
        # Apply these functions to ONLY the numeric features
        # Just in case, impute NULL values with mean

        numeric_pipeline = Pipeline(steps=[
        # ('impute', SimpleImputer(strategy = 'median')),
        ('impute', KNNImputer()),
        ('scale', MinMaxScaler())
        ])

        # Apply these functions to ONLY to the categorical features
        # Then, apply One Hot Encoder

        categorical_pipeline = Pipeline(steps = [
            ('impute', SimpleImputer(strategy = 'most_frequent')),
            ('one-hot', OneHotEncoder(handle_unknown = 'ignore', sparse = False)),
        ])
        
        # Runs the above numerical and categorical pipeline functions and create final processor

        full_processor = ColumnTransformer(transformers=[
        ('number', numeric_pipeline, numerical_features),
        ('category', categorical_pipeline, categorical_features)
        ])
        
        return full_processor

In [ ]:
clean_data_pointer = clean_data(train_data, 'SalePrice')
c = clean_data_pointer.remove_outliers_iqr(train_data, 'SalePrice')
interested_cols = clean_data_pointer.feature_selection(c, 'SalePrice')

train_data = train_data[interested_cols]
test_data = test_data[interested_cols[:-1]]

train_data.shape, test_data.shape

Inner Boundary 1 = 3937.5
Inner Boundary 2 = 340037.5
Outer Boundary 1 = -122100.0
Outer Boundary 2 = 466075.0
Data before Removing Outliers = (1460, 81)
Data after Removing Outliers = (1448, 81)
Number of Outliers removed = 12


((1460, 24), (1459, 23))

In [ ]:
# For model training split the train data
X = train_data.drop(['SalePrice'], axis=1)
Y = train_data['SalePrice']


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

features = train_data.columns[:-1]
target = 'SalePrice'

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical = ['object']
numer_features = x_train.select_dtypes(include=numerics)
cat_features = x_train.select_dtypes(include=categorical)

# Defining the features
numerical_features = numer_features.columns
categorical_features = cat_features.columns
target = 'SalePrice'
features = train_data.columns[0:-1]

# Call earlier pipeline 
enc = My_encoder()
full_processor = enc.encoder_pipeline(numerical_features, categorical_features)

# Transform data - OHE, min max scaler for both test and train
train_x = full_processor.fit_transform(x_train)
test_x = full_processor.transform(x_test)

#### Random Forest

In [ ]:
# Initialize Random Forest (RF) model creation
rfc = RandomForestRegressor()

# Number of trees
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# number of features at every split
max_features = ['sqrt']

#max depth
max_depth = [int(x) for x in np.linspace(20, 200, num = 10)]
max_depth.append(None)

# create random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth
}

# Random search of parameters
rfc_random = RandomizedSearchCV(
                                estimator = rfc, 
                                param_distributions = random_grid, 
                                n_iter = 3, 
                                cv = 2, 
                                verbose = 2, 
                                random_state = 66,
                                n_jobs = -1) # -1 indicates to use the computer's maximum available CPU threads to run

# # Fit the model
rfc_random.fit(train_x, y_train)
# best params = {'n_estimators': 1200, 'max_features': 'sqrt', 'max_depth': 20}

#print results
print(rfc_random.best_params_)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
{'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 20}


#### Evaluate RandomForest

In [ ]:
rfc_model = RandomForestRegressor(**(rfc_random.best_params_))
#rfc_model = RandomForestClassifier(**rfc_parameters)

# fit the model
rfc_model.fit(train_x, y_train)

y_pred_rfc = rfc_model.predict(test_x)

#print("Accuracy = ", metrics.accuracy_score(y_test, y_pred_rfc))
print("R2_Score = ", r2_score(y_test, y_pred_rfc)) # jump in score to 83%
print("MAE = ", mean_absolute_error(y_test, y_pred_rfc))

R2_Score =  0.8861025853011948
MAE =  17995.159306792644


#### XGBoost

In [ ]:
import xgboost as xgb
clf_xgb = xgb.XGBRegressor()

param_dist = {'n_estimators':       [50,105], # number of trees
              'learning_rate':      scipy.stats.uniform(0.01, 0.50),
              'max_depth':          [4, 10, 20],
              'min_child_weight':   [1, 2, 3, 4, 5],
              # WARNING: THE BELOW ONES MAKE IT WORSE
              # 'subsample':          scipy.stats.uniform(0.3, 0.6),
              # 'colsample_bytree':   scipy.stats.uniform(0.5, 0.4),
              # 'num_parallel_tree':  [2],
              # 'gamma':              [0.3],
              # 'alpha':              [0.5]
             }

xgb_cv = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = 2,  
                         n_iter = 3, # will use 6 fits
                         error_score = 0, 
                         verbose = 3, 
                         random_state = 66,
                         n_jobs = -1) # -1 indicates to use the computer's maximum available CPU threads to run

xgb_cv.fit(train_x, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[23:19:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=2, error_score=0, estimator=XGBRegressor(), n_iter=3,
                   n_jobs=-1,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fca9e2a5750>,
                                        'max_depth': [4, 10, 20],
                                        'min_child_weight': [1, 2, 3, 4, 5],
                                        'n_estimators': [50, 105]},
                   random_state=66, verbose=3)

#### Evalaute XGBoost

In [ ]:
#xgb_cv = xgb.XGBRegressor(**(xgb_cv.best_params_))

#xgb_cv.fit(train_x, y_train)

y_pred_xg = xgb_cv.predict(test_x)

#y_pred = rfc_model.predict(test_x)

#print("Accuracy = ", metrics.accuracy_score(y_test, y_pred_xg))
print("R2-Score = ", r2_score(y_test, y_pred_xg))
print("MAE = ", mean_absolute_error(y_test, y_pred_xg))

R2-Score =  0.9060642541436846
MAE =  17871.30094178082


#### Neural Network Model

In [ ]:
!nvidia-smi

Mon Oct 10 23:19:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |    386MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# Check Pytorch installation
print(torch.__version__, torch.cuda.is_available())

1.12.1+cu113 True


In [ ]:
print("Is CUDA available? :", torch.cuda.is_available())
device = torch.device("cuda")

Is CUDA available? : True


In [ ]:
# Transform ndarray to tensor
x_train_tensor = torch.tensor(train_x).float()
x_test_tensor = torch.tensor(test_x).float()
y_test_tensor = torch.tensor(y_test.values).float()
y_train_tensor = torch.tensor(y_train.values).float()

# Set input dimension
n_input = x_train_tensor.shape[1]
# Set hidden dimension
n_hidden = 32
# Set output dimension
n_output = 1

#x_train_tensor.shape, x_test_tensor.shape, y_test_tensor.shape, y_train_tensor.shape

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_input,n_output,n_hidden):
        super(NeuralNetwork, self).__init__()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(n_input,n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden,n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden,n_output),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# Allocate device to use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Transfer model to GPU
model = NeuralNetwork(n_input,n_output,n_hidden).to(device)
# Transfer data to GPU
x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
x_test_tensor = x_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=23, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [ ]:
train_tensor = data.TensorDataset(x_train_tensor, y_train_tensor)
test_tensor = data.TensorDataset(x_test_tensor, y_test_tensor)
# mini-batch size
batch_size = 32
# Train dataloader
train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_tensor, batch_size=batch_size, shuffle=True)

lr = 0.1
criterion = nn.L1Loss() # measures MAE loss
optimizer = optim.Adam(model.parameters(),lr=lr)
num_epochs = 20
learn_process = np.zeros((0,2))
metric_name='r2'

In [ ]:
for epoch in range(num_epochs):
    train_loss = 0

    for x, y in tqdm(train_loader): #tqm pytorch DataLoader iterator
        score=[]
        loss_all=[]

        # Transfer data to GPU
        x = x.to(device)
        y = y.to(device)
        
        # Initialize grad
        optimizer.zero_grad()
        
        # Predict in train data
        y_pred = model(x)
        
        # Calculating loss
        #loss = torch.sqrt(criterion(y_pred, y)) # <- RMSE
        loss = criterion(y_pred, y)
        
        # Calculating grad
        #backward
        loss.backward()
        
        # Modify params
        optimizer.step()
    
    learn_process = np.vstack((learn_process,np.array([epoch,loss.item()])))
    print(f'Epoch {epoch} loss: {loss.item():.5f})')

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 0 loss: 94013.33594)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 1 loss: 55932.30859)


...

Epoch 17 loss: 47386.13672)


  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 18 loss: 44533.25391)


  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 19 loss: 62159.32812)


In [ ]:
model.eval()
test_losses = []
num_correct = 0

for inputs, labels in test_loader:

    test_output = model(inputs)
    loss = criterion(test_output, labels)
    #loss = loss_function(test_output, labels)
    test_losses.append(loss.item())
    
    preds = torch.round(test_output.squeeze())
    correct_tensor = preds.eq(labels.float().view_as(preds))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    
print("Test Loss: {:.4f}".format(np.mean(test_losses)))
print("Test Accuracy: {:.2f}".format(num_correct/len(test_loader.dataset)))  

Test Loss: 58704.6867
Test Accuracy: 0.00


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


#### Using TensorFLow Neural Net Model

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(256, kernel_initializer='normal', activation='relu'),
    keras.layers.Dense(128, kernel_initializer='normal', activation='relu'),
    keras.layers.Dense(128, kernel_initializer='normal', activation='relu'),
    keras.layers.Dense(1, kernel_initializer='normal', activation='linear')])

model.compile(optimizer= 'adam', loss = tf.keras.losses.MeanAbsoluteError(), metrics = ['accuracy'])

model.fit(train_x, y_train, epochs = 20)

Epoch 1/20
37/37 [==============================] - 2s 3ms/step - loss: 181434.6406 - accuracy: 0.0000e+00
Epoch 2/20
37/37 [==============================] - 0s 3ms/step - loss: 181105.7188 - accuracy: 0.0000e+00
Epoch 3/20
37/37 [==============================] - 0s 3ms/step - loss: 177578.5469 - accuracy: 0.0000e+00
Epoch 4/20
37/37 [==============================] - 0s 3ms/step - loss: 160387.4375 - accuracy: 0.0000e+00
Epoch 5/20
37/37 [==============================] - 0s 3ms/step - loss: 106733.3438 - accuracy: 0.0000e+00
Epoch 6/20
37/37 [==============================] - 0s 3ms/step - loss: 33730.9336 - accuracy: 0.0000e+00
Epoch 7/20
37/37 [==============================] - 0s 3ms/step - loss: 28608.3965 - accuracy: 0.0000e+00
Epoch 8/20
37/37 [==============================] - 0s 3ms/step - loss: 28398.9805 - accuracy: 0.0000e+00
Epoch 9/20
37/37 [==============================] - 0s 3ms/step - loss: 28138.5156 - accuracy: 0.0000e+00
Epoch 10/20
37/37 [======================

#### Evaluating the model

In [ ]:
train_loss, train_acc = model.evaluate(train_x, y_train)
print('Train Loss:',train_loss)

test_loss, test_acc = model.evaluate(test_x, y_test) # change this to encoder array
print('Test Loss:', test_loss)

37/37 [==============================] - 0s 2ms/step - loss: 26173.9219 - accuracy: 0.0000e+00
Train Loss: 26173.921875
10/10 [==============================] - 0s 2ms/step - loss: 27625.3652 - accuracy: 0.0000e+00
Test Loss: 27625.365234375


In [ ]:
Prediction_Error = pd.DataFrame({
    'Model': ['Random Forest Regressor', 'XGBoost Regressor', 'Neural Network'],
    'MAE Error': [mean_absolute_error(y_test, y_pred_rfc), mean_absolute_error(y_test, y_pred_xg), test_loss]})
Prediction_Error

,Model,MAE Error
0,Random Forest Regressor,17995.159307
1,XGBoost Regressor,17871.300942
2,Neural Network,26173.921875
